In [ ]:
import numpy as np
from scipy.stats import mode
import sys
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
from scipy.io import loadmat
import time
from helper import *

In [5]:
def innerproduct(X,Z=None):
    
    # Computes the inner-product matrix.
    # Input:
    # X: nxd data matrix with n vectors (rows) of dimensionality d
    # Z: mxd data matrix with m vectors (rows) of dimensionality d
    #
    # Output:
    # Matrix G of size nxm
    # G[i,j] is the inner-product between vectors X[i,:] and Z[j,:]
    
    if Z is None: # case when there is only one input (X)
        Z=X;
    G = np.dot(X,(Z.T))
    return G    

In [6]:
def l2distance(X,Z=None):
    
    # Computes the Euclidean distance matrix.
    # Input:
    # X: nxd data matrix with n vectors (rows) of dimensionality d
    # Z: mxd data matrix with m vectors (rows) of dimensionality d
    #
    # Output:
    # Matrix D of size nxm
    # D(i,j) is the Euclidean distance of X(i,:) and Z(j,:)
    
    
    if Z is None:
        Z=X;

    n,d1=X.shape
    m,d2=Z.shape
    if (d1==d2):
#   "Dimensions of input vectors must match!"
        G = innerproduct(X,Z)        
#         Using broadcasting method to calculate D where shapes are incompatible

        D = np.sum(np.square(X)[:,np.newaxis,:], axis=2) - 2*G + np.sum(np.square(Z), axis=1)
        D = D.clip(min=0)
        D_dist = np.sqrt(D)
        
    return D_dist


In [ ]:
xTr,yTr,xTe,yTe=loaddata("faces.mat")

plt.figure(figsize=(11,8))
plotfaces(xTr[:9, :])

In [13]:
def findknn(xTr,xTe,k):
    """
    function [indices,dists]=findknn(xTr,xTe,k);
    
    Finds the k nearest neighbors of xTe in xTr.
    
    Input:
    xTr = nxd input matrix with n row-vectors of dimensionality d
    xTe = mxd input matrix with m row-vectors of dimensionality d
    k = number of nearest neighbors to be found
    
    Output:
    indices = kxm matrix, where indices(i,j) is the i^th nearest neighbor of xTe(j,:)
    dists = Euclidean distances to the respective nearest neighbors
    
    """
    E_dist = l2distance(xTr,xTe)    
    dists = np.sort(E_dist,0)[0:k]    
    indices = np.argsort(E_dist,0)[0:k]
    return indices,dists

In [14]:
def accuracy(truth,preds):
    """
    function output=accuracy(truth,preds)         
    Analyzes the accuracy of a prediction against the ground truth
    
    Input:
    truth = n-dimensional vector of true class labels
    preds = n-dimensional vector of predictions
    
    Output:
    accuracy = scalar (percent of predictions that are correct)
    """
    
    truth = truth.flatten()
    preds = preds.flatten()
    count=0
    for i in range(len(truth)): 
        if truth[i]!=preds[i]:
            count+=1
    return np.abs(1 - (count/len(truth)))

In [2]:
def knnclassifier(xTr,yTr,xTe,k):
    """
    function preds=knnclassifier(xTr,yTr,xTe,k);
    
    k-nn classifier 
    
    Input:
    xTr = nxd input matrix with n row-vectors of dimensionality d
    xTe = mxd input matrix with m row-vectors of dimensionality d
    k = number of nearest neighbors to be found
    
    Output:
    
    preds = predicted labels, ie preds(i) is the predicted label of xTe(i,:)
    """
    # fix array shapes
    yTr = yTr.flatten()
    our_labels=[]
    indices,dists = findknn(xTr,xTe,k)
    for i in indices:
        our_labels.append(yTr[i])
    our_labels = np.array(our_labels)
    print(our_labels)
    
    from collections import defaultdict
    d=defaultdict(int)
    for l in our_labels:
        d[tuple(l)] += 1
    preds = np.array(list(max(d)))
    return preds

In [ ]:
print("Face Recognition: (1-nn)")
xTr,yTr,xTe,yTe=loaddata("faces.mat") # load the data
t0 = time.time()
preds = knnclassifier(xTr,yTr,xTe,1)
result=accuracy(yTe,preds)
t1 = time.time()
print("You obtained %.2f%% classification acccuracy in %.4f seconds\n" % (result*100.0,t1-t0))